## Setup: Finnhub API Key

This notebook uses real market data from [Finnhub](https://finnhub.io/) (free tier: 60 API calls/min).

**To use real data:**
1. Get a free API key at https://finnhub.io/register
2. Set environment variable: `export FINNHUB_API_KEY=your_key_here`
3. Or enter the key when prompted

**Without a key:** The notebook falls back to synthetic data generation.

# Portfolio Hedging via PCA
Find eigenvectors of covariance and hedge exposures.

In [ ]:
import numpy as np, matplotlib.pyplot as plt

# Import Finnhub helper for real market data
try:
    from python.finnhub_helper import fetch_historical_simulation, get_finnhub_api_key, create_orderbook_from_quote
    FINNHUB_AVAILABLE = True
except Exception as e:
    print(f"Finnhub helper not available: {e}")
    FINNHUB_AVAILABLE = False

np.random.seed(42)
T=3000; N=5
f=np.cumsum(np.random.randn(T)*0.5)
X=np.vstack([f+0.2*np.cumsum(np.random.randn(T))+0.1*np.random.randn(T) for _ in range(N)]).T
C=np.cov(X,rowvar=False)
val,vec=np.linalg.eigh(C)
idx=np.argsort(val)[::-1]; val,vec=val[idx],vec[:,idx]
w=vec[:,0]/np.linalg.norm(vec[:,0])
y=X[:,0]; f1=X@w
beta=np.cov(y,f1)[0,1]/np.var(f1)
res=y-beta*f1
plt.figure(); plt.plot(res); plt.title('Hedged'); plt.show()
